In [ ]:
import os
from pathlib import Path
from deepeval_evaluation.rag_deepeval_utils import load_goldens, build_deepeval_testcases_from_goldens, evaluate_with_deepeval
from src.document_ingestion.data_ingestion import ChatIngestor
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# 1) Configuration
PROJECT_ROOT = Path.cwd()
GOLDENS_PATH = PROJECT_ROOT / "goldens.json"
SAMPLE_PATH = PROJECT_ROOT / "sample.pdf"
FAISS_BASE = os.getenv("FAISS_BASE", "faiss_index")
FAISS_INDEX_NAME = os.getenv("FAISS_INDEX_NAME", "index")
SESSION_ID = "deepeval_test"
build_index_from_scratch = True
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [ ]:
# 2) Build a FAISS session from local docs or skip if index has been already created
if build_index_from_scratch:
	paths_for_upload = [open(SAMPLE_PATH, "rb")]
	ci = ChatIngestor(temp_base="data", faiss_base=FAISS_BASE, use_session_dirs=True, session_id=SESSION_ID)
	retriever = ci.build_retriever(paths_for_upload, k=5, enable_ocr=False)
	session_id = ci.session_id
	print("Built new session:", session_id)
else:
	session_id = SESSION_ID

In [ ]:
# 3) Load goldens
goldens = load_goldens(str(GOLDENS_PATH))

In [ ]:
# 4) Run the RAG evaluation (build test cases)
test_cases = build_deepeval_testcases_from_goldens(goldens=goldens, session_id=session_id, k=5)

In [ ]:
# 5) Run DeepEval
results = evaluate_with_deepeval(test_cases=test_cases)